## Computer Vision Learning Notebook

Author: Jackson Guthrie.
Purpose: Classify a form-based long document into various document types using computer vision techniques.

Environment: cv-learning-dev from `environment.yml` in this repo.


In [6]:
!conda install pytorch=2.6.0 --yes

Solving environment: | 

In [1]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
from PIL import Image
import requests
import io
from datasets import load_dataset
from itertools import islice

/Users/jackson/anaconda3/envs/cv-learning-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/jackson/anaconda3/envs/cv-learning-env/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/jackson/anaconda3/envs/cv-learning-env/lib/python3.10/runpy.py", line 86,

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [3]:
# Download example image
dataset = load_dataset("aharley/rvl_cdip", split = "test", streaming=True, trust_remote_code=True)
example = next(islice(dataset, 100, 101))
image = example["image"] # Load image
true_label_id = example["label"]

print(image, true_label_id)

<PIL.Image.Image image mode=L size=754x1000 at 0x17EB7ADA0> 0


In [4]:
# Load processor and model
processor = AutoImageProcessor.from_pretrained("microsoft/dit-base-finetuned-rvlcdip")
model = AutoModelForImageClassification.from_pretrained("microsoft/dit-base-finetuned-rvlcdip")

# Move model to GPU
model.to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


ValueError: Due to a serious vulnerability issue in `torch.load`, even with `weights_only=True`, we now require users to upgrade torch to at least v2.6 in order to use the function. This version restriction does not apply when loading files with safetensors.
See the vulnerability report here https://nvd.nist.gov/vuln/detail/CVE-2025-32434

In [ ]:
# The image is already a PIL object, so we ensure it's in RGB format
# then process and move it to the device
inputs = processor(images=image.convert("RGB"), return_tensors="pt").to(device)

In [ ]:
# Perform inference
with torch.no_grad(): # Disable gradient calculations for faster inference
    outputs = model(**inputs)

logits = outputs.logits

# Get the predicted class index
predicted_class_idx = logits.argmax(-1).item()

In [ ]:

# Print the results
print("\n--- Results ---")
print(f"True Class:      {dataset.features['label'].names[true_label_id]}")
print(f"Predicted Class: {model.config.id2label[predicted_class_idx]}")


--- Results ---


NameError: name 'dataset' is not defined